# Creating the Data Files

Below you see the code to create the data files and retrieve data from the Docker container test_sshd and test_sshd2.

See also the example code on [AppliAR.jl](https://www.appligate.nl/AppliAR.jl/stable/chapter4/).

##### Prerequisites

- A processor unit of at least 3 cores.
- Ubuntu 20.04.
- [Julia 1.5.3](https://www.appligate.nl/BAWJ/stable/appendix/#Install-Julia).
- [IJulia package](https://www.appligate.nl/BAWJ/stable/appendix/#Install-IJulia) added.
- The packages Bukdu, Distributed, DataFrames, and Query added.
- The package AppliAR added (`add https://github.com/rbontekoe/AppliAR.jl`).
- [Docker](https://www.appligate.nl/BAWJ/stable/appendix/#Install-Docker) installed.
- The two docker containers `test_sshd` and `test_sshd2` created according to the course [BAWJ chapter 13](https://www.appligate.nl/BAWJ/stable/chapter13/).
- The GitHub project TestAppliAR cloned (`git clone https://github.com/rbontekoe/TestAppliAR.git`).
- The packages AppliSales, AppliGeneralLedger, AppliAR, and Query have also been added to the two containers.
- The data files have been created by running the notebook ar.ipynb.

### Activate the local environment in the TestAppliAR folder

The application uses the packages:
- [AppliSales](https://github.com/rbontekoe/AppliSales.jl)
- [AppliGeneralLegder](https://www.appligate.nl/AppliGeneralLedger.jl/)
- [AppliAR](https://www.appligate.nl/AppliAR.jl/stable/) (use: add https://github.com/rbontekoe/AppliAR.jl)
- [DataFrames](https://github.com/JuliaData/DataFrames.jl)
- [Rocket](https://github.com/biaslab/Rocket.jl)
- [Query](https://github.com/queryverse/Query.jl)

In [1]:
using Pkg; Pkg.activate(".");

 Activating environment at `~/projects/TestAppliAR/Project.toml`


### Load [Rocket.jl](https://github.com/biaslab/Rocket.jl), the actor package

In [2]:
using Rocket

### Start the containers test_sshd and test_sshd2

See section 3 `creating the containers` of the course: [BAWJ](https://www.appligate.nl/BAWJ/stable/chapter13/)

In [3]:
cmd = `docker restart test_sshd`; run(cmd)
cmd = `docker restart test_sshd2`; run(cmd);

test_sshd
test_sshd2


Process(`docker restart test_sshd2`, ProcessExited(0))

### Allow distributed computation

In [4]:
using Distributed

### Connect to the containes test_sshd (General Ledger) and test_sshd2 (Accounts Receivable)

In [5]:
# get the is-addresses of the containers
cmd_sshd = `docker inspect -f '{{ .NetworkSettings.IPAddress }}' test_sshd`; ip_sshd = read(cmd_sshd, String)
cmd_sshd2 = `docker inspect -f '{{ .NetworkSettings.IPAddress }}' test_sshd2`; ip_sshd2 = read(cmd_sshd2, String)
# strip \n
ip_sshd = ip_sshd[1:length(ip_sshd)-1] 
ip_sshd2 = ip_sshd2[1:length(ip_sshd2)-1]

"172.17.0.3"

In [6]:
addprocs([("rob@" * ip_sshd, 1)]; exeflags=`--project=$(Base.active_project())`, tunnel=true, dir="/home/rob")
addprocs([("rob@" * ip_sshd2, 1)]; exeflags=`--project=$(Base.active_project())`, tunnel=true, dir="/home/rob")

1-element Array{Int64,1}:
 3

### Assign the two processes

In [7]:
gl_pid = procs()[2] # general ledger
ar_pid = procs()[3] # accounts receivable (invoices/bankstatements)

3

### Load the packages

The packages must also be available in the containers test_sshd and test_sshd2.

In [8]:
@everywhere using AppliSales
@everywhere using AppliGeneralLedger
@everywhere using AppliAR
@everywhere using Query

### Load the actors

See: [actors.jl](https://www.appligate.nl/AppliAR.jl/stable/chapter4/#actors.jl)

In [9]:
include("./src/actors.jl");

In [10]:
sales_actor = SalesActor()
ar_actor = ARActor(ar_pid)
gl_actor = GLActor(gl_pid)
stm_actor = StmActor();

### Start the application

                           StmActor
                              |
                              | BankStatement(s)
                              ↓       
       SalesActor -------> ARActor -------> GLActor
                  Order(s)    ↑    Entry(s)    ↑
                              ↓                ↓
                            Store            Store

The SalesActor sends a message with three orders to the ARActor (Accounts Receivable Actor). The ARActor creates three invoices and sends the journal entries to the GLActor (General Ledger Actor). 

In [20]:
AppliSales.process() # The three orders

3-element Array{AppliSales.Order,1}:
 AppliSales.Order("10768281901969063295", AppliSales.Organization("7098413851735848268", "Scrooge Investment Bank", "1180 Seven Seas Dr", "FL 32830", "Lake Buena Vista", "USA"), AppliSales.Training("LS", Dates.DateTime("2019-08-30T00:00:00"), 2, "Learn Smiling", 1000.0), "PO-456", "Scrooge McDuck", "scrooge@duckcity.com", ["Scrooge McDuck"])
 AppliSales.Order("1901249410867120201", AppliSales.Organization("2782853483338105883", "Duck City Chronicals", "1185 Seven Seas Dr", "FL 32830", "Lake Buena Vista", "USA"), AppliSales.Training("LS", Dates.DateTime("2019-08-30T00:00:00"), 2, "Learn Smiling", 1000.0), "DD-001", "Mickey Mouse", "mickey@duckcity.com", ["Mini Mouse", "Goofy"])
 AppliSales.Order("6082499958350773097", AppliSales.Organization("7855438390493228369", "Donalds Hardware Store", "1190 Seven Seas Dr", "FL 32830", "Lake Buena Vista", "USA"), AppliSales.Training("LS", Dates.DateTime("2019-08-30T00:00:00"), 2, "Learn Smiling", 1000.0), "", "Do

In [21]:
subscribe!(from(["START"]), sales_actor); # Start the application

┌ Info: GLActor completed!
└ @ Main /home/rob/projects/TestAppliAR/src/actors.jl:54
┌ Info: GLActor completed!
└ @ Main /home/rob/projects/TestAppliAR/src/actors.jl:54
┌ Info: GLActor completed!
└ @ Main /home/rob/projects/TestAppliAR/src/actors.jl:54
┌ Info: ARActor Completed!
└ @ Main /home/rob/projects/TestAppliAR/src/actors.jl:41
┌ Info: SalesActor completed!
└ @ Main /home/rob/projects/TestAppliAR/src/actors.jl:24


### Process the two bankstatements

In [22]:
subscribe!(from(["READ_STMS"]), stm_actor);

stms = BankStatement[BankStatement(Dates.Date("2020-01-15"), "Duck City Chronicals Invoice A1002", "NL93INGB", 2420.0), BankStatement(Dates.Date("2020-01-15"), "Donalds Hardware Store Bill A1003", "NL39INGB", 1210.0)]


┌ Info: GLActor completed!
└ @ Main /home/rob/projects/TestAppliAR/src/actors.jl:54
┌ Info: GLActor completed!
└ @ Main /home/rob/projects/TestAppliAR/src/actors.jl:54
┌ Info: ARActor Completed!
└ @ Main /home/rob/projects/TestAppliAR/src/actors.jl:41
┌ Info: StmActor completed!
└ @ Main /home/rob/projects/TestAppliAR/src/actors.jl:13


### Display General Leger Accounts Receivable (1300)

You can play with the accountids of the general ledger:
- 1150 Bank
- 1300 Accounts Receivable
- 4000 VAT
- 8000 Sales

In [23]:
using DataFrames

In [24]:
gl_account_id = 1150

1150

In [25]:
gl = @fetchfrom gl_pid AppliGeneralLedger.read_from_file("./test_ledger.txt")
df = gl |> @filter(_.accountid == gl_account_id) |> DataFrame
df[[:invoice_nbr, :customerid, :debit, :credit, :descr]]

,invoice_nbr,customerid,debit,credit,descr
,String,String,Float64,Float64,String
1,A1002,Duck City Chronicals,2420.0,0.0,Learn Smiling
2,A1003,Donalds Hardware Store,1210.0,0.0,Learn Smiling


### Retrieve Aging Report (Unpaid invoices)

In [26]:
ar = @fetchfrom ar_pid report()
ar |> DataFrame

,id_inv,csm,inv_date,amount,days
,String,String,Dates…,Float64,Dates…
1,A1001,Scrooge Investment Bank,"Date(""2021-01-22"")",1210.0,Day(0)


### Remove the data files from the containers

The SalesActor creates the orders only one time. To repeat the process, the data files have to be remove otherwise you get dublicated records. Keep in mind that also this notebook has to restarted (shutdown and start), unless you don't run cells 4 untill 12 again.

In [18]:
cmd = `ssh rob@172.17.0.2 \ rm test\*`
run(cmd)

Process(`ssh rob@172.17.0.2 ' rm' 'test*'`, ProcessExited(0))

In [19]:
cmd = `ssh rob@172.17.0.3 \ rm test\* invoicenbr.txt`
run(cmd)

Process(`ssh rob@172.17.0.3 ' rm' 'test*' invoicenbr.txt`, ProcessExited(0))